TEST WITH NDCG

In [ ]:
# Function to create subsets of data
def create_data_subsets(data, group_size):
    grouped_data = [data.iloc[i : i + group_size] for i in range(0 , data.shape[0], group_size)]
    return grouped_data

def calculate_ndcg_for_groups(groups, model):
    ndcg_scores = []
    for group in tqdm(groups, total=len(groups), unit=" items"):
        tokenized_X_train_group = group['ad_creative_bodies'].apply(tokenize_and_remove_stopwords)
        X_train_word2vec_group = generate_doc_vectors(tokenized_X_train_group, word2vec_model, 100)

        y_group_pred = model.predict(X_train_word2vec_group)

        # Assuming lower CPI values are more relevant
        relevance_scores = 1 / y_group_pred

        # Normalize relevance scores (optional)
        #relevance_scores = (relevance_scores - np.min(relevance_scores)) / (np.max(relevance_scores) - np.min(relevance_scores))

        y_group_true = 1 / group['cpi'].values

        try:
            ndcg = ndcg_score([y_group_true], [relevance_scores])
            ndcg_scores.append(ndcg)
        except Exception as e:
            print(y_group_true)
            print(relevance_scores)

    return ndcg_scores

group_size = 4
test_data_subsets = create_data_subsets(test_data, group_size)

ndcg_scores = calculate_ndcg_for_groups(test_data_subsets, gb_regressor)

PRINT NDCG SCORES

In [ ]:
# Combine NDCG scores across groups (simple average in this example)
average_ndcg_across_groups = sum(ndcg_scores) / len(ndcg_scores)

print(f"Average NDCG Across Groups: {round(average_ndcg_across_groups,2)}")